<a href="https://colab.research.google.com/github/sm241/machine-learnig/blob/main/ml_final3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
# $ conda install -c conda-forge scikit-surprise

In [3]:
pip install surprise

     |████████████████████████████████| 11.8MB 6.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617649 sha256=320def66f454ad895b88f1dff8b52e5f7b0487b54f7e8f5f81f1ac412f86ff53
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [5]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, accuracy

import warnings; warnings.simplefilter('ignore')

몇 가지 추천 알고리즘을 구현하고 최종 추천 시스템 모델을 완성시킬 노트북이다.
구현 순서는 다음과 같다. 단순 추천 시스템, 콘텐츠 기반 추천, 메타 데이터 기반 추천, 키워드 추천, 협업 필터링 마지막으로 하이브리드 추천 시스템을 간단하게 구축할 것이다. 

먼저, 일반 대중이 좋아할 가능성이 높은 더 대중적이고 비평적인 영화를 추천하는 방식으로 사용자 개인화는 하지 못할 가장 간단한 시스템을 먼저 만들어 보겠습니다. 

In [85]:
md = pd.read_csv('/content/movies_metadata.csv.zip')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [12]:
print('////////////////////// Before ////////////////////// \n', md['genres'].head(), '\n////////////////////// Before //////////////////////')

# dictionary안에 담겨있는 Genre 정보를 List 형태로 세팅
# 1. md['genres'].fillna('[]') : genres 컬럼에 null 값을 '[](빈 리스트 값)'으로 채워넣음
# 2. apply(literal_eval) : literal_eval를 사용하여 String으로 되어있는 값을 List & Dictionary로 사용할 수 있게 변환
# 3. apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) : x가 list인 경우 안에 들어있는 dictionary 중 name에 해당하는 값을 list에 담음
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

print('////////////////////// After ////////////////////// \n', md['genres'].head(), '\n////////////////////// After //////////////////////')

////////////////////// Before ////////////////////// 
 0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object 
////////////////////// Before //////////////////////
////////////////////// After ////////////////////// 
 0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object 
////////////////////// After //////////////////////


다음으로 목록에 있는 영화의 최소 95%보다 더 많은 표를 가진 영화들을 골라 차트화시켜보겠습니다. 이를 위하여 weighted rating 공식을 이용하였습니다. 


Weighted Rating (WR) =  (𝑣𝑣+𝑚.𝑅)+(𝑚𝑣+𝑚.𝐶) 

v : 영화에 대한 평가 수

m : 차트에 표시되어야 하는 최소 평가 수

R : 영화의 평점

C : 전체 영화에 대한 평균 점수

In [13]:
print('vote ::: \n', md[['vote_count', 'vote_average']].head())
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

vote ::: 
    vote_count  vote_average
0      5415.0           7.7
1      2413.0           6.9
2        92.0           6.5
3        34.0           6.1
4       173.0           5.7


5.244896612406511

In [14]:
# 총 45460개의 영화 중 상위 5%는 2273번째
print(vote_counts.sort_values(ascending=False)[2273:2274])

# quantile는 데이터를 크기대로 정렬하였을 때 분위수를 구하는 함수. quantile(0.95)는 상위 5%에 해당하는 값을 찾는 것
m = vote_counts.quantile(0.95)
m

11561    434
Name: vote_count, dtype: int64


434.0

In [15]:
print('release_date ::: \n', md['release_date'].head())

# pd.to_datetime
# errors : {‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
# If ‘raise’, then invalid parsing will raise an exception
# If ‘coerce’, then invalid parsing will be set as NaT
# If ‘ignore’, then invalid parsing will return the input

# 'release_date'를 split해서 year만 추출
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

print('year ::: \n', md['year'].head())

release_date ::: 
 0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object
year ::: 
 0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: object


In [16]:
# 평가 수가 상위 5%인(434보다 큰) 데이터 추출
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [17]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [18]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [19]:
# Weighted Rating 상위 250개의 영화 
qualified = qualified.sort_values('wr', ascending=False).head(250)

top 15를 차트로 잘 나타나는지 확인해봅니다. 

In [20]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


크리스토퍼 놀란 감독의 영화인 인셉션, 다크 나이트, 인터스텔라가 차트의 맨 위에 나타나는 것을 볼 수 있습니다. 

이제 특정 장르에 대해 차틀르 작성하는 함수를 정의하겠습니다. 이를 위해 백분위수를 95에서 85로 완화하겠습니다. 

In [21]:
# stack() : stack이 (위에서 아래로 길게, 높게) 쌓는 것이면, unstack은 쌓은 것을 옆으로 늘어놓는것(왼쪽에서 오른쪽으로 넓게) 라고 연상이 될 것
# reset_index() : 기존의 행 인덱스를 제거하고 인덱스를 데이터 열로 추가
s = md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
print(s.head(10))

gen_md = md.drop('genres', axis=1).join(s)
print(gen_md.head(10))

0    Animation
0       Comedy
0       Family
1    Adventure
1      Fantasy
1       Family
2      Romance
2       Comedy
3       Comedy
3        Drama
Name: genre, dtype: object
   adult                              belongs_to_collection  ...  year      genre
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  ...  1995  Animation
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  ...  1995     Comedy
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  ...  1995     Family
1  False                                                NaN  ...  1995  Adventure
1  False                                                NaN  ...  1995    Fantasy
1  False                                                NaN  ...  1995     Family
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...  ...  1995    Romance
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...  ...  1995     Comedy
3  False                                                NaN  ...  1995     Comedy
3  

In [23]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

이제 로맨스 영화의 탑 차트를 불러와보겠습니다. 

In [24]:
build_chart('Romance').head(15)


,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


종합 탑 차트에 거의 등장하지 못했던 로맨스 영화들이 장르에 맞게 차트화된 것을 확인 할 수 있습니다. 

지금부터는 사용자의 취향에 맞게 추천하는 시스템들을 구축해보겠습니다. 우선, 영화 개요 및 태그 라인을 이용한 콘텐츠 기반 추천 시스템을 구축한 뒤 영화 메타 데이터 즉, 영화의 출연진, 제작진, 키워드 및 장르를 기반으로 추천하는 시스템을 만들어보겠습니다. 시스템들을 구축하기 전에 데이터 셋을 더 작게 사용하는 작업부터 하겠습니다. 

In [26]:
links_small = pd.read_csv('/content/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
links_small.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [27]:
# Drop a row by index : 19730, 29503, 33587 행은 이상한 데이터들(md.iloc[19730], md.iloc[29503], md.iloc[33587])
md = md.drop([19730, 29503, 35587])

In [28]:
#Check EDA Notebook for how and why I got these indices.
md['id'] = md['id'].astype('int')

In [29]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

기존에 사용했던 영화 테이터의 1/5에 해당하는 9099편의 영화 데이터 셋입니다. 

먼저 영화 내용과 태그 라인을 사용한 영화 내용 기반 추천 시스템을 만들어보겠습니다. 

In [30]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

smd['description'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: description, dtype: object

In [31]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [32]:
print(tfidf_matrix[10])

  (0, 237880)	0.14118434571637967
  (0, 264587)	0.15239448296082492
  (0, 149397)	0.1346268358004416
  (0, 184592)	0.13757548758984955
  (0, 192607)	0.15239448296082492
  (0, 12515)	0.15239448296082492
  (0, 59410)	0.15239448296082492
  (0, 204261)	0.15239448296082492
  (0, 260369)	0.15239448296082492
  (0, 221301)	0.15239448296082492
  (0, 51585)	0.15239448296082492
  (0, 15732)	0.15239448296082492
  (0, 213978)	0.15239448296082492
  (0, 142130)	0.15239448296082492
  (0, 256956)	0.14583697304488685
  (0, 255009)	0.15239448296082492
  (0, 73255)	0.15239448296082492
  (0, 232769)	0.15239448296082492
  (0, 51809)	0.15239448296082492
  (0, 255952)	0.15239448296082492
  (0, 154810)	0.14583697304488685
  (0, 184597)	0.14583697304488685
  (0, 264425)	0.13213378623513553
  (0, 213982)	0.15239448296082492
  (0, 10509)	0.15239448296082492
  :	:
  (0, 259727)	0.15239448296082492
  (0, 237762)	0.08073572734141646
  (0, 192601)	0.13213378623513553
  (0, 12513)	0.13757548758984955
  (0, 59409)	0.14

In [33]:
tfidf_matrix.shape

(9099, 268124)

두 영화 사이의 유사성을 코사인 유사도를 사용하여 숫자 수량으로 계산해보겠습니다. 

𝑐𝑜𝑠𝑖𝑛𝑒(𝑥,𝑦)=𝑥.𝑦⊺/||𝑥||.||𝑦||

In [34]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [35]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

이제 데이터 셋의 모든 영화는 각 영화에 대해 쌍 단위 코사인 유사도 메트릭스를 갖습니다. 이제, 코사인 유사도 점수를 기반으로 가장 유사한 30개의 영화를 반환하는 함수를 작성해보겠습니다. 

In [36]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

print(titles.head(), indices.head())

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [37]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

이제 모든 준비가 끝났습니다. 몇 개의 영화에 대한 추천을 받아보고 얼마나 그 추천이 좋은지 알아보겠습니다. 

In [38]:
get_recommendations('Toy Story').head(10)

2502               Toy Story 2
7535               Toy Story 3
6193    The 40 Year Old Virgin
2547           Man on the Moon
6627              Factory Girl
4702    What's Up, Tiger Lily?
889      Rebel Without a Cause
6554    For Your Consideration
4988          Rivers and Tides
1599                 Condorman
Name: title, dtype: object

In [39]:
get_recommendations('Inception').head(10)

5239                              Cypher
141                                Crumb
6398                         Renaissance
653                            Lone Star
1703                               House
4739                    The Pink Panther
319                                 Cobb
2828    What Ever Happened to Baby Jane?
8867                     Pitch Perfect 2
979          Once Upon a Time in America
Name: title, dtype: object

다크 나이트의 경우 시스템에서 베트맨 영화로 식별한 후 다른 베트맨 영화를 추천할 수 있지만, 영화의 평점 및 인기를 결정하는 출연지, 제작진, 감독 및 장르와 같은 사항들을 고려하지 않습니다. 따라서 다음 스텝은 이들을 고려하는 시스템들을 하나씩 구현하겠습니다. 

먼저, 메타 데이터 기반 추천 시스템을 구축하기전에 현재 데이터 셋을 제작진 및 키워드 데이터 셋과 병합을 해야합니다. 

In [41]:
credits = pd.read_csv('/content/credits.csv.zip')
keywords = pd.read_csv('/content/keywords.csv.zip')

In [42]:
credits['crew'][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [43]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [44]:
md.shape

(45463, 25)

In [45]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [46]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

이제, 출연진, 제작진, 장르 및 크레딧이 모두 하나의 데이터 프레임에 존재합니다. 

In [47]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [48]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [49]:
smd['director'] = smd['crew'].apply(get_director)

In [50]:
# 출연진 중 상위에 노출되는 3명만 추출(주연배우들만)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >= 3 else x)


In [51]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


다음 작업은 모든 영화에 대해 장르, 감독, 주연 배우 및 키워드로 구성된 메타 데이터 텀프를 만들고, count vectorizer를 사용하여, 카운트 매트릭스를 만들고 코사인 유사도를 계산하여 가장 유사한 영화를 반환하는 것입니다. 

장르 및 크레딧 테이터를 준비 할 때 아래와 같은 작업을 추가적으로 해줍니다.

1.모든 기능에서 스트립 스페이스 및 소문자로 변환. 이런 식으로 우리 엔진은 Johnny Depp와 Johnny Galecki를 혼동하지 않습니다.

2.감독은 2배로 언급하여은 전체 캐스트에 비해 가중치를 더 줍니다.


In [52]:
# 출연진의 이름에서 공백 삭제
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [53]:
# 감독의 이름에서 공백 삭제 및 2번 언급
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x, x])

키워드를 사용하기 전에 데이터 셋에서 나타나는 모든 키워드의 빈도 수를 계산합니다. 

In [54]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [55]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

키워드는 2번 이상 언급된 이들만 사용할 것이고, 어근이 동일한 것으로 간주되도록 모든 단어를 stem으로 변환해줍니다. 

In [56]:
# 2번 이상 등장한 키워드만 추출
s = s[s > 1]

In [58]:
# 어근 추출을 통해 동일 의미&다른 형태의 단어(dogs&dog, imaging&image 등)를 동일한 단어로 인식
stemmer = SnowballStemmer('english')
print("dogs의 어근 : ", stemmer.stem('dogs'))
print("dog의 어근 : ", stemmer.stem('dog'))

dogs의 어근 :  dog
dog의 어근 :  dog


In [59]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [60]:
# 키워드의 어근을 찾아서 공백 제거 후 세팅
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [61]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [62]:
count = CountVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [63]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [65]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

모든 작업을 완료했으므로 다시한번 다크나이트가 어떤 추천을 받는지 확인해보겠습니다. 

In [66]:
get_recommendations('The Dark Knight').head(10)

8031                 The Dark Knight Rises
6218                         Batman Begins
7659            Batman: Under the Red Hood
6623                          The Prestige
1134                        Batman Returns
8927               Kidnapping Mr. Heineken
5943                              Thursday
1260                        Batman & Robin
2085                             Following
9024    Batman v Superman: Dawn of Justice
Name: title, dtype: object

이 추천은 크리스토퍼 놀란 감독의 다른 영화를 인식하고 베트맨을 중심으로 추천을 잘 넣은 것 같습니다. 

이제, 위 추천 시스템에서 인기도와 평점들도 관여하는 시스템을 만들어보겠습니다. 유사도 점수를 기준으로 상위 25개의 영활르 선정하고 60번째 백분위 수 영화의 투표를 계산한 뒤 이것을 m값으로 이용하여 IMDB의 공식을 사용하여 각 영화의 가중치 등급을 계산하는 작업을 하겠습니다. 

In [67]:
def improved_recommendations(title):
    print(title)
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    print(movie_indices)

    movies = smd.iloc[movie_indices][['title','vote_count','vote_average','year']]
#     print(movies)
    
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull())]
#     print(qualified)
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    print(qualified)
    return qualified

잘 만들어졌는지 확인해보겠습니다. 

In [68]:
improved_recommendations('The Dark Knight')

The Dark Knight
6981
[8031, 6218, 7659, 6623, 1134, 8927, 5943, 1260, 2085, 9024, 2448, 5098, 8026, 8727, 9121, 5809, 7362, 7561, 7582, 4021, 2754, 217, 628, 2272, 3647]
                                   title  vote_count  ...  year        wr
6623                        The Prestige        4510  ...  2006  7.758148
8031               The Dark Knight Rises        9263  ...  2012  7.494595
6218                       Batman Begins        7511  ...  2005  7.376814
7659          Batman: Under the Red Hood         459  ...  2010  6.455414
1134                      Batman Returns        1706  ...  1992  6.325180
2085                           Following         363  ...  1998  6.135364
7561                         Harry Brown         351  ...  2009  5.895522
9024  Batman v Superman: Dawn of Justice        7189  ...  2016  5.674090
8026                  Bullet to the Head         490  ...  2013  5.221088
1260                      Batman & Robin        1447  ...  1997  4.441087

[10 rows x 5 co

,title,vote_count,vote_average,year,wr
6623,The Prestige,4510,8.0,2006,7.758148
8031,The Dark Knight Rises,9263,7.6,2012,7.494595
6218,Batman Begins,7511,7.5,2005,7.376814
7659,Batman: Under the Red Hood,459,7.6,2010,6.455414
1134,Batman Returns,1706,6.6,1992,6.325180
2085,Following,363,7.2,1998,6.135364
7561,Harry Brown,351,6.7,2009,5.895522
9024,Batman v Superman: Dawn of Justice,7189,5.7,2016,5.674090
8026,Bullet to the Head,490,5.2,2013,5.221088
1260,Batman & Robin,1447,4.2,1997,4.441087


비추천을 받아야 마땅할 영화들이 보이지만 어느정도는 성공한 모습입니다. 


콘텐츠 기반 엔진은 특정 영화와 유사한 영화만을 제안할 수 있어서 새로운 취향을 포착할 수 없으며 장르 전체에 대한 권장 사항을 제공할 수 없다는 단점이 존재합니다. 또한, 조회하는 사람이 누구인지는 관계없이 해당 영화에 대한 동일한 추천을 해줍니다. 따라서 조금 더 개인적인 추천을 해주는 협어 필터링이라는 시스템을 만들어보겠습니다. 

협업 필터링은 특정 제품이나 서비스를 사용해본 나와 유사한 사용자 정보가 해당 제품이나 서비스를 내가 얼마나 좋아하는지 예측하는데 사용할 수 있다는 아이디어를 기반으로 작동합니다. 여기서 협업 필터링 자체를 구현할 수 없으므로 Surprise 라이브러리를 사용하여, RMSE를 최소화하고 추천하는 시스템을 만들어보겠습니다.  

In [69]:
# surprise 라이브러리의 Reader
reader = Reader()

In [71]:
ratings = pd.read_csv('/content/ratings_small.csv.zip')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [72]:
data = Dataset.load_from_df(ratings[['userId', 'movieId','rating']], reader)
# data.split(n_folds=5)

trainset = data.build_full_trainset()
testset = trainset.build_testset()

In [73]:
svd = SVD()
# evaluate(svd, data, measures=['RMSE', 'MAE'])
svd.fit(trainset)
predictions = svd.test(testset)
accuracy.rmse(predictions)


RMSE: 0.6418


0.6418330852339637

rmse가 0.6473으로 적당히 나왔으므로 데이터 셋을 학습하고 예측해보겠습니다. 

사용자를 50,000명으로 선택하고 부여한 평점을 확인해보겠습니다. 

In [74]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [75]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.715518876215276, details={'was_impossible': False})

ID가 302인 영화의 경우는 2,501로 추정됩니다. 이 시스템은 영화가 무엇인지는 상관하지 않고 지정된 영화 ID를 기준으로 작동하면서 다른 사용자가 영화를 어떻게 예측했는지에 따라 등급을 예측하는 시스템입니다. 

다음 작업은 위에서 구축한 콘텐츠 기반 및 협업 필터링 기반 엔진에서 구현한 기술들을 통하하는 간단한 하이브리드 추천 시스템을 구축해보겠습니다. 

입력 : 사용자 ID 및 영화 제목


출력 : 특정 사용자의 예상 등급을 기준으로 정렬된 유사한 영화.


In [76]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [77]:
id_map = pd.read_csv('/content/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')


In [78]:
indices_map = id_map.set_index('id')


In [79]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title','vote_count','vote_average','year','id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

 작업을 완료했으니 영화 아바타에 대한 추천을 받아보갰습니다. (사용자의 ID를 달리하여)

In [80]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.064241
974,Aliens,3282.0,7.7,1986,679,3.015366
1011,The Terminator,4208.0,7.4,1984,218,3.006598
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.980404
7705,Alice in Wonderland,8.0,5.4,1933,25694,2.919139
2834,Predator,2129.0,7.3,1987,106,2.888602
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,2.875950
2014,Fantastic Planet,140.0,7.6,1973,16306,2.820924
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.728216
3060,Sinbad and the Eye of the Tiger,39.0,6.3,1977,11940,2.712663


In [81]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
2014,Fantastic Planet,140.0,7.6,1973,16306,3.492718
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.487311
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.348666
1011,The Terminator,4208.0,7.4,1984,218,3.280199
4017,Hawk the Slayer,13.0,4.5,1980,25628,3.226517
2132,Superman II,642.0,6.5,1980,8536,3.190078
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.177752
7705,Alice in Wonderland,8.0,5.4,1933,25694,3.102916
2834,Predator,2129.0,7.3,1987,106,3.086018
1668,Return from Witch Mountain,38.0,5.6,1978,14822,3.063593


하이브리드 추천 시스템의 경우는 영화는 동일하지만 사용자마다 다른 추천을 하는 시스템으로 더욱 개인화되고 맞춤화되는것을 확인할 수 있습니다. 

# https://www.kaggle.com/alsojmc/movie-recommender-systems/notebook?select=credits.csv 참조


